In [22]:
import numpy as np
import utils 
import pandas as pd 
from sklearn.impute import * 

Okay the goal is to generate causal data on a very small dataset where

A -> B 

A -> C -> D

with the following equation set 

$$ A \sim N(1,1) $$ 
$$ B = A + A^2 + N(0,1) $$ 
$$ C = A^3 + N(0,1) $$ 
$$ D = C^2 + 1 + N(0,1) $$ 

In [2]:
utils.generate_full_data(1000, True, "test1")

array([[-2.92772059e-01,  5.43301504e-02,  7.44621110e-01,
         3.11351255e-01],
       [-9.36995372e-01, -3.42950429e-01, -1.38058378e+00,
         5.16373575e+00],
       [ 1.43754748e+00,  4.38254314e+00,  2.95831418e+00,
         9.41467130e+00],
       ...,
       [ 2.16281987e+00,  4.26684901e+00,  8.82894637e+00,
         7.77354650e+01],
       [-9.90969658e-01,  5.81559064e-01, -2.07607246e+00,
         4.97567744e+00],
       [ 2.86238481e+00,  1.19284200e+01,  2.44347021e+01,
         5.97114542e+02]])

In [3]:
data = np.loadtxt("./gen_data/test1.csv", delimiter = ",")

In [4]:
data.shape

(1000, 4)

In [7]:
mcar_data = utils.v_induce_missing_data(data, 0.2)

In [8]:
mcar_data.shape

(1000, 4)

so next I am checking approx what percentage of the data is missing, I should also look for any complete cases next 

In [11]:
np.sum(np.isnan(mcar_data)) / (data.shape[0]* data.shape[1])

0.20775

In [14]:
mcar2 = pd.DataFrame(mcar_data)
mcar2.dropna(axis=0, inplace=True)

so here I am testing to see how many would result in a complete case 

In [17]:
print(f"Number complete case: {mcar2.shape}")
print(f"Percentage complete cases: {mcar2.shape[0]/1000}") 

Number complete case: (394, 4)
Percentage complete cases: 0.394


now i want to save the data that i created for later reuse, so ill save the mcar and the complete cases 

In [ ]:
# commented out so they're not rerun in run all 
# np.savetxt("./gen_data/mcar_test1.csv", mcar_data, delimiter=",")

In [ ]:

# mcar2.to_csv("./gen_data/complete_cases_test1.csv")

next task would be imputation 

In [25]:
imp = SimpleImputer(strategy="mean")

mean_imp = imp.fit_transform(mcar_data)

In [ ]:
# np.savetxt("./gen_data/mean_imp_test1.csv", mean_imp, delimiter=",")